In [24]:
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load("ru_core_news_sm")
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, mean_squared_error
from sklearn.model_selection import train_test_split
from json import load


In [25]:
not_discriptors = list()
with open('../../Data/Words/word_pairs_not_disc.txt', 'r', encoding='utf-8') as txt:
    for i in txt:
        not_discriptors.append(i.replace('\n', ''))

with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    events = load(js)

with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discriptors = load(js)


In [26]:
dict_of_disc = {'expession': [], 'is_discr': []}
for subject in discriptors.values():
    for d in sum(subject.values(), []):
        dict_of_disc['expession'].append(nlp(d).vector)
        dict_of_disc['is_discr'].append(True)

for subject in not_discriptors:
    dict_of_disc['expession'].append(nlp(subject).vector)
    dict_of_disc['is_discr'].append(False)



In [27]:
features = np.array(dict_of_disc['expession'])
labels = np.array(list(map(int, dict_of_disc['is_discr'])))


In [28]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=71)


In [29]:
import tensorflow as tf

model_class = tf.keras.models.Sequential()
model_class.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*1.5), activation='relu',input_shape=(X_train[0].shape[0],), name='input'))
model_class.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*2), activation='relu', name='hidden_layer_2'))
model_class.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*3), activation='relu', name='hidden_layer_3'))
model_class.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*2), activation='sigmoid', name='hidden_layer_4', use_bias=True))
model_class.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*1.5), activation='sigmoid', name='hidden_layer_5', use_bias=True))
model_class.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*0.5), activation='sigmoid', name='hidden_layer_6', use_bias=True))
model_class.add(tf.keras.layers.Dense(1, name='output', activation='sigmoid'))
model_class.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 144)               13968     
                                                                 
 hidden_layer_2 (Dense)      (None, 192)               27840     
                                                                 
 hidden_layer_3 (Dense)      (None, 288)               55584     
                                                                 
 hidden_layer_4 (Dense)      (None, 192)               55488     
                                                                 
 hidden_layer_5 (Dense)      (None, 144)               27792     
                                                                 
 hidden_layer_6 (Dense)      (None, 48)                6960      
                                                                 
 output (Dense)              (None, 1)                

In [30]:
model_class.compile(
    loss='mse',  # Функция потерь
    optimizer='Adam',  # Оптимизатор
    metrics=[  # Метрики
        'mse',  # Если у объекта назначено имя, то можно вызвать объект с его помощью
        # tf.keras.metrics.TrueNegatives
        # tf.keras.metrics.Precision()
    ]
)


In [31]:
model_class.fit(
    X_train,  # Набор входных данных
    y_train,  # Набор правильных ответов
    validation_split=0.2,  # Этот параметр автоматически выделит часть обучающего набора на валидационные данные. В данном случа 20%
    epochs=40,  # Процесс обучения завершится после 10 эпох
    # Набор данных будет разбит на пакеты (батчи) по 8 элементов набора в каждом.
    batch_size=2
)


Epoch 1/40
319/319 [==============================] - 2s 3ms/step - loss: 0.1240 - mse: 0.1240 - val_loss: 0.0661 - val_mse: 0.0661
Epoch 2/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0740 - mse: 0.0740 - val_loss: 0.0836 - val_mse: 0.0836
Epoch 3/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0733 - mse: 0.0733 - val_loss: 0.0722 - val_mse: 0.0722
Epoch 4/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0731 - mse: 0.0731 - val_loss: 0.0880 - val_mse: 0.0880
Epoch 5/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0617 - mse: 0.0617 - val_loss: 0.0917 - val_mse: 0.0917
Epoch 6/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0541 - mse: 0.0541 - val_loss: 0.0808 - val_mse: 0.0808
Epoch 7/40
319/319 [==============================] - 1s 2ms/step - loss: 0.0686 - mse: 0.0686 - val_loss: 0.1025 - val_mse: 0.1025
Epoch 8/40
319/319 [==============================] - 1s 2ms/step - loss: 0.

In [32]:
predict = model_class.predict(X_test)


9/9 [==============================] - 0s 2ms/step


In [33]:
mean_squared_error(predict, y_test)


0.05298755684476419

In [34]:
predictA = model_class.predict(features)
print('Accuracy')
print(accuracy_score(labels, predictA > 0.5))
print('Confusion matrix')
print(confusion_matrix(labels, predictA > 0.5))
print('Precision, Recall, F\n', classification_report(
    labels, predictA > 0.5))


34/34 [==============================] - 0s 1ms/step
Accuracy
0.9586854460093897
Confusion matrix
[[324  34]
 [ 10 697]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.97      0.91      0.94       358
           1       0.95      0.99      0.97       707

    accuracy                           0.96      1065
   macro avg       0.96      0.95      0.95      1065
weighted avg       0.96      0.96      0.96      1065



In [35]:
print('Accuracy')
print(accuracy_score(y_test, predict > 0.98))
print('Confusion matrix')
print(confusion_matrix(y_test, predict > 0.98))
print('Precision, Recall, F\n', classification_report(
    y_test, predict > 0.98))


Accuracy
0.3595505617977528
Confusion matrix
[[ 96   0]
 [171   0]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.36      1.00      0.53        96
           1       0.00      0.00      0.00       171

    accuracy                           0.36       267
   macro avg       0.18      0.50      0.26       267
weighted avg       0.13      0.36      0.19       267



c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [36]:
tn, fp, fn, tp = confusion_matrix(y_test, predict > 0.9).ravel()
(tn, fp, fn, tp)


(84, 12, 3, 168)

In [37]:
model_class.predict(np.array([nlp('рекомендации, чтобы поддерживать внутреннюю').vector]))


1/1 [==============================] - 0s 20ms/step


array([[0.9560471]], dtype=float32)

In [38]:
model_class.predict(
    np.array([nlp('чтобы поддерживать внутреннюю').vector]))


1/1 [==============================] - 0s 20ms/step


array([[0.9554036]], dtype=float32)

In [39]:
model_class.predict(
    np.array([nlp('поддерживать внутреннюю структуру').vector]))


1/1 [==============================] - 0s 18ms/step


array([[0.95606273]], dtype=float32)

In [40]:
model_class.save('../../PipeLines/Classifications/checker_is_discriptor_spacy_vectorize.h5')
